A neural network consist of cnn layer (Kim,2014) and BiLSTM.

Source: https://github.com/jojonki/cnn-for-sentence-classification





In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [82]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout
from keras.layers import Conv2D, MaxPooling1D
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM, Bidirectional

In [83]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0,converters={'body': eval})
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [84]:
len(labels)

78

In [85]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 300


In [86]:
freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

زبان


In [87]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [88]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 225345
129280


In [89]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [90]:
X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# print(vecX[0])

train:  (12935, 300) 
test:  (4312, 300) 
val:  (4312, 300) 
y_tain: (12935, 78)


In [91]:
embd_dim = 300


# ***If the word2vec model is not generated before, we should run the next block.***

In [ ]:
embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
embed_model.save('word2vec_model')

# ***Otherwise, we can run the next block.***

In [92]:
embed_model=gensim.models.Word2Vec.load('word2vec_model')

In [93]:
word2vec_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  if word in embed_model.wv.vocab:
        embedding_vector =embed_model[word]
  
        # words not found in embedding index will be all-zeros.
        word2vec_embd_w[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [94]:
def calculating_class_weights(y_true):
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])
    return weights

class_weights=calculating_class_weights(y_train)
print(class_weights)

[[  0.51329365  19.30597015]
 [  0.51773135  14.5993228 ]
 [  0.51773135  14.5993228 ]
 [  0.50174554 143.72222222]
 [  0.50949267  26.83609959]
 [  0.50190129 131.98979592]
 [  0.51806312  14.34035477]
 [  0.50397413  63.40686275]
 [  0.50334656  75.20348837]
 [  0.51264268  20.27429467]
 [  0.51727585  14.97106481]
 [  0.50873122  29.13288288]
 [  0.51702774  15.18192488]
 [  0.5016677  150.40697674]
 [  0.5018234  137.60638298]
 [  0.51731723  14.93648961]
 [  0.51496934  17.20079787]
 [  0.51868634  13.87875536]
 [  0.51268331  20.2109375 ]
 [  0.50773277  32.82994924]
 [  0.51603766  16.08830846]
 [  0.51447777  17.76785714]
 [  0.50444583  56.73245614]
 [  0.51706908  15.14637002]
 [  0.50801194  31.70343137]
 [  0.51723448  15.00580046]
 [  0.5174828   14.79977117]
 [  0.51752421  14.76598174]
 [  0.51492834  17.24666667]
 [  0.51636727  15.77439024]
 [  0.50713558  35.53571429]
 [  0.50989436  25.76693227]
 [  0.51464152  17.57472826]
 [  0.51789718  14.46868009]
 [  0.50877124

In [ ]:
def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss

In [123]:
from keras.layers import LSTM
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[word2vec_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    embd_sentence = Permute((2,1))(embd_sentence)
    embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # cnn
    cnn = Conv2D(1, 
                 kernel_size=(5, sentence_maxlen),
                 activation='relu')(embd_sentence)
    cnn =  Lambda(lambda x: K.sum(x, axis=3))(cnn)
    cnn = MaxPooling1D(3)(cnn)
    # cnn = Lambda(lambda x: K.sum(x, axis=2))(cnn)
    print(cnn.shape)
    lstm_cell=500
    
    bilstm = Bidirectional(
        LSTM(lstm_cell),
        merge_mode='concat'
    )(cnn)
    
    hidden1=Dense(400,activation="relu")(bilstm)
    hidden2=Dense(300,activation="relu")(hidden1)
    hidden3=Dense(200,activation="relu")(hidden2)
    hidden4=Dense(150,activation="relu")(hidden3)
    out = Dense(len(labels), activation='sigmoid')(hidden4)
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
   
    # model.compile(optimizer=sgd, loss=get_weighted_loss(class_weights),metrics=["accuracy"])
    model.compile(optimizer=sgd, loss='binary_crossentropy',metrics=["accuracy"]) 
    # model.compile(optimizer=sgd, loss='kl_divergence',metrics=["accuracy"])

    return model

model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)
print(model.summary())


(None, 98, 1)
Model: "sentence_claccification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   [(None, 300)]             0         
_________________________________________________________________
shared_embd (Embedding)      (None, 300, 300)          67603500  
_________________________________________________________________
permute_30 (Permute)         (None, 300, 300)          0         
_________________________________________________________________
lambda_60 (Lambda)           (None, 300, 300, 1)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 296, 1, 1)         1501      
_________________________________________________________________
lambda_61 (Lambda)           (None, 296, 1)            0         
_________________________________________________________________
max_pooling1d_30 (MaxPooling 

In [124]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) # Model stop training after 5 epoch where validation loss didnt decrease
mc = ModelCheckpoint('best_cnn_bilstm.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True) #You save model weight at the epoch where validation loss is minimal
model.fit(X_train, y_train, batch_size=32,epochs=250,verbose=1,validation_data=(X_val, y_val),callbacks=[es,mc])#you can run for 250 epoch btw model will stop after 5 epoch without better validation loss

Epoch 1/250
405/405 [==============================] - 773s 2s/step - loss: 0.5065 - accuracy: 0.0105 - val_loss: 0.1097 - val_accuracy: 0.0065

Epoch 00001: val_loss improved from inf to 0.10975, saving model to best_cnn_bilstm.h5
Epoch 2/250
405/405 [==============================] - 768s 2s/step - loss: 0.1097 - accuracy: 0.0215 - val_loss: 0.1082 - val_accuracy: 0.0255

Epoch 00002: val_loss improved from 0.10975 to 0.10816, saving model to best_cnn_bilstm.h5
Epoch 3/250
405/405 [==============================] - 766s 2s/step - loss: 0.1095 - accuracy: 0.0124 - val_loss: 0.1078 - val_accuracy: 0.0299

Epoch 00003: val_loss improved from 0.10816 to 0.10779, saving model to best_cnn_bilstm.h5
Epoch 4/250
405/405 [==============================] - 767s 2s/step - loss: 0.1068 - accuracy: 0.0192 - val_loss: 0.1076 - val_accuracy: 0.0306

Epoch 00004: val_loss improved from 0.10779 to 0.10755, saving model to best_cnn_bilstm.h5
Epoch 5/250
405/405 [==============================] - 760s 

KeyboardInterrupt: ignored

# ***If the model is generated before:***

In [ ]:
# model = load_model('best_cnn_4fc_with_binary.h5')
model.save('best_cnn_4fc_with_binary.h5')

OSError: ignored

In [ ]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [ ]:
print(pred[0])#example

[0.02787289 0.03535512 0.03507045 0.01073387 0.02101916 0.01036984
 0.03654274 0.01384887 0.01231143 0.02728426 0.03476822 0.01982093
 0.03218392 0.01028025 0.01014689 0.03465104 0.03288615 0.03589937
 0.02665889 0.01809195 0.03094345 0.02960333 0.01322055 0.03627309
 0.01992357 0.03516915 0.03567177 0.0350053  0.03109333 0.03355917
 0.01755849 0.0226225  0.02826291 0.03595707 0.02145717 0.03536102
 0.03588098 0.01796857 0.01599711 0.01196662 0.03485608 0.02687773
 0.02075732 0.03521562 0.0351398  0.01115128 0.01256019 0.03498513
 0.03404218 0.01950714 0.01206458 0.01157954 0.01379335 0.02380064
 0.03137466 0.01332781 0.01720527 0.01582348 0.0340479  0.03565708
 0.03470814 0.01504529 0.0363718  0.03594434 0.02301514 0.03683951
 0.01281136 0.02489835 0.03440848 0.03594753 0.03181466 0.03491029
 0.02923766 0.01342568 0.01220992 0.03421348 0.01810268 0.02877665]


In [ ]:
y_pred=[]
for l in pred:
  temp=[]
  for value in l:
    if value>=0.5:
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score

print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy= 0.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       134
           1       0.00      0.00      0.00       131
           2       0.00      0.00      0.00       154
           3       0.00      0.00      0.00        15
           4       0.00      0.00      0.00        90
           5       0.00      0.00      0.00        11
           6       0.00      0.00      0.00       150
           7       0.00      0.00      0.00        43
           8       0.00      0.00      0.00        24
           9       0.00      0.00      0.00       109
          10       0.00      0.00      0.00       150
          11       0.00      0.00      0.00        79
          12       0.00      0.00      0.00       138
          13       0.00      0.00      0.00        21
          14       0.00      0.00      0.00        17
          15       0.00      0.00      0.00       157
          16       0.00      0.00      0.00       123
          17 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
